In [ ]:
!pip3 install pathway bokeh --quiet # This cell may take a few seconds to execute.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
import pathway as pw
import bokeh.plotting
import panel as pn
###########reading the csv
df = pd.read_csv('dataset.csv')
##################
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df['LotID_Encoded'], unique_lots = pd.factorize(df['SystemCodeNumber'])
df = df.sort_values(by=['Timestamp']).reset_index(drop=True)



# -------------------------
# Schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    LotID_Encoded: int
    Occupancy: int
    Capacity: int

# -------------------------

# Save the selected columns to a CSV file for streaming or downstream processing
df[["Timestamp", "Occupancy", "Capacity","LotID_Encoded"]].to_csv("parking_stream.csv", index=False)


# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

pn.extension()


# Windowed Transformation
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.LotID_Encoded),
        window=pw.temporal.tumbling(timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        LotID_Encoded=pw.reducers.max(pw.this.LotID_Encoded)
    )
    .with_columns(
        price=10 + .7*(pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

# -------------------------
# Bokeh Plotting Function
def price_plotter(source, lot_id):
    fig = bokeh.plotting.figure(
        height=300,
        width=800,
        title=f"Dynamic Price  for  {lot_id}",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=5, color="red")
    return fig

# -------------------------
# Individual Lot Plots
plots = []

# Manually specify all unique LotID_Encoded values (if you know them)

for lot_id, lot_name in enumerate(unique_lots):
    lot_table = delta_window.filter(pw.this.LotID_Encoded == lot_id)
    viz = lot_table.plot(lambda source, name=lot_name: price_plotter(source, name), sorting_col="t")
    plots.append(viz)

# Combine into a scrollable vertical layout
pn.Column(*plots).servable()

ModuleNotFoundError: No module named 'pathway'

In [ ]:

pw.run()
